### module

In [196]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_log_error

In [197]:
train_data = pd.read_csv('./data/train_data.csv')
test_data = pd.read_csv('./data/test_data.csv')
target = train_data['y']
del train_data['id']
del train_data['y']
del test_data['id']

### preprocessing

In [198]:
data = pd.concat([train_data, test_data])

In [199]:
data.columns

Index(['種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称', '最寄駅：距離（分）',
       '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造', '用途',
       '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等'],
      dtype='object')

In [200]:
data.head(2)

,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,1,２ＬＤＫ,55,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2017年第４四半期,改装済,NaN
1,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,5,１Ｋ,20,...,住宅,NaN,NaN,NaN,商業地域,80.0,500.0,2017年第４四半期,未改装,NaN


In [201]:
# 種類
kind_counts = data['種類'].value_counts()
print(kind_counts)
kind_nan = data['種類'].isnull().sum()
print('count of nan == ',kind_nan)
# labelEncoder
data['種類'] = LabelEncoder().fit_transform(data['種類'])

中古マンション等     176330
宅地(土地と建物)    137154
宅地(土地)        77185
林地              427
農地               92
Name: 種類, dtype: int64
count of nan ==  0


In [202]:
# 地域
data['地域'] = data['地域'].fillna('空',inplace = True)
data['地域'] = LabelEncoder().fit_transform(data['地域'])

In [203]:
# 市区町村コード
del data['市区町村コード']
# 都道府県
del data['都道府県名']

In [204]:
# 市区町村名

In [205]:
data.head(2)

,種類,地域,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,土地の形状,間口,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,0,0,千代田区,飯田橋,飯田橋,1,２ＬＤＫ,55,NaN,NaN,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,2017年第４四半期,改装済,NaN
1,0,0,千代田区,飯田橋,飯田橋,5,１Ｋ,20,NaN,NaN,...,住宅,NaN,NaN,NaN,商業地域,80.0,500.0,2017年第４四半期,未改装,NaN


In [206]:
data.columns

Index(['種類', '地域', '市区町村名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
       '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '前面道路：方位',
       '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時点', '改装',
       '取引の事情等'],
      dtype='object')

In [207]:
# 市区町村名
data['市区町村名'].value_counts()
# 区　or 市　or else
areas = []
for area in data['市区町村名']:
    c = 0
    if '区' in area:
        c = 1
    elif '市' in area:
        c = 2
    else:
        c = 3
    areas.append(c)
data['市区村区別'] = pd.Series(areas)

In [208]:
# 地区名
# data['地区名'].isnull().sum() #nan is 246
# 3A1R
area_names = []
data['地区名'].fillna('空', inplace = True)
for area_name in data['地区名']:
    c = 0
    if ('六本木') and ('赤坂') and ('青山') and ('麻布') in area_name:
        c = 1
    else:
        c = 2
    area_names.append(c)
data['3A1R'] = pd.Series(area_names)

In [209]:
# 地域詳細
data['地域詳細'] = data['市区町村名'] + data['地区名']
data['市区町村名'] = LabelEncoder().fit_transform(data['市区町村名'])
del data['市区町村名']
del data['地区名']

In [210]:
# station
'''
外部から各駅の利用者人数を使う
'''
data['最寄駅：名称'].value_counts()

八王子        3631
大泉学園       3245
新小岩        3192
金町         2498
三鷹         2367
           ... 
青海(東京)        1
東京テレポート       1
遊園地西          1
武蔵小杉          1
桜ケ丘           1
Name: 最寄駅：名称, Length: 656, dtype: int64

In [211]:
data['station_dis'] = data['最寄駅：距離（分）'].replace('30分?60分', 45).replace('1H?1H30', 75).replace('1H30?2H', 105).replace('2H?', 120)

In [213]:
data['station_dis'] = data['station_dis'].astype(int)
data['station_dis'] = data['station_dis'].fillna(data['station_dis'].mean()) 

TypeError: can only concatenate str (not "int") to str

0

In [191]:
data.head()

,種類,地域,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,土地の形状,間口,延床面積（㎡）,建築年,...,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,市区村区別,3A1R,地域詳細,station_dis
0,0,0,飯田橋,1,２ＬＤＫ,55,NaN,NaN,NaN,昭和59年,...,商業地域,80.0,600.0,2017年第４四半期,改装済,NaN,1,2,千代田区飯田橋,1
1,0,0,飯田橋,5,１Ｋ,20,NaN,NaN,NaN,平成15年,...,商業地域,80.0,500.0,2017年第４四半期,未改装,NaN,1,2,千代田区飯田橋,5
2,0,0,飯田橋,3,１ＬＤＫ,45,NaN,NaN,NaN,平成24年,...,商業地域,80.0,500.0,2017年第３四半期,未改装,NaN,1,2,千代田区飯田橋,3
3,0,0,飯田橋,5,１Ｒ,20,NaN,NaN,NaN,平成15年,...,商業地域,80.0,500.0,2017年第３四半期,未改装,NaN,1,2,千代田区飯田橋,5
4,2,0,飯田橋,3,NaN,80,ほぼ台形,6.8,330,昭和61年,...,商業地域,80.0,500.0,2017年第２四半期,NaN,NaN,1,2,千代田区飯田橋,3


In [177]:
# modeling

In [ ]:
train = data.iloc[:len(train_data), :]
test = data.iloc[len(train_data):, :]
cv = KFold(n_splits=5, shuffle=True, random_state=4)

In [ ]:
for tr_idx, val_idx in cv.split(train_data):
    x_train, x_val = train.iloc[tr_idx], train.iloc[val_idx]
    y_train, y_val = y[tr_idx], y[val_idx]
    
    # Datasetに入れて学習させる
    train_set = lgb.Dataset(x_train, y_train)
    val_set = lgb.Dataset(x_val, y_val, reference=train_set)
        
    # Training
    model = lgb.train(params, train_set, num_boost_round=30000, early_stopping_rounds=1000, 
                      valid_sets=[train_set, val_set], verbose_eval=500)
    
    # 予測したらexpで元に戻す
    test_pred = np.exp(model.predict(test))
    # 0より小さな値があるとエラーになるので補正
    test_pred = np.where(test_pred < 0, 0, test_pred)
    pred += test_pred / 5  #  5fold回すので

    oof = np.exp(model.predict(x_val))
    oof = np.where(oof < 0, 0, oof)
    rmsle = np.sqrt(mean_squared_log_error(np.exp(y_val), oof))
    c += 1
    print("{}回目の思考".format(c))
    print(f"RMSLE : {rmsle}")
    score += rmsle/ 5